In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import warnings
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
search = "Restaurants in new york, usa"
urls = f"https://www.google.com/maps/search/{search.replace(' ', '+')}"
print(urls)

https://www.google.com/maps/search/Restaurants+in+new+york,+usa


In [29]:
def load_companies(url):
    options = webdriver.ChromeOptions()
    print("Getting business info", url)
    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s, options=options)
    driver.get(url)
    time.sleep(5)
    # panel_xpath = '//*[@class="m6QErb Hk4XGb QoaCgb KoSBEe tLjsW "][4]'
    # scrollable_div = driver.find_element(By.XPATH, panel_xpath)
    # link = driver.find_element(By.LINK_TEXT, "More reviews")
    # link.click()
    # time.sleep(5)
    # # scrolling
    # flag = True
    # i = 0
    columns = ["Name", "Rating", "Reviews", "Address", "Category", "Time"]
    df = pd.DataFrame(columns=columns)
    # while i < 1:
    #     print(f"Scrolling to page {i + 2}")
    #     driver.execute_script(scrollable_div)
    #     time.sleep(7)
    #     i += 1
    #     # print(df)
    # try:
    #     all_reviews_button = driver.find_element(By.XPATH, "//button[contains(@aria-label, 'All reviews')]")
    #     all_reviews_button.click()
    #     time.sleep(10)
    #     print(driver.page_source)
    # except Exception as e:
    #     print("No reviews found or failed to click on All Reviews button.")
    #     driver.quit()
    #     return None
    print(driver.page_source)
    driver.close()
    return df

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time

In [60]:
restaurant_name = "Restaurant Above"

In [79]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.get('https://www.google.com/maps')

# Search for the restaurant
search_box = driver.find_element(By.NAME, 'q')
search_box.send_keys(restaurant_name)
search_box.send_keys(Keys.ENTER)

time.sleep(2)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'More reviews ')]"))).click()
time.sleep(2)
c = driver.page_source
panel = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]'
wait = WebDriverWait(driver, 10)
scrollable_div = wait.until(EC.presence_of_element_located((By.XPATH, panel)))

for i in range(0,5):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)", scrollable_div)
    time.sleep(5)



In [80]:
sp = BeautifulSoup(c, "html")

In [81]:
sp

<html itemscope="" itemtype="http://schema.org/Place" jstcache="0" lang="en"><head> <link href="/maps/preview/opensearch.xml?hl=en" rel="search" title="Google Maps" type="application/opensearchdescription+xml"/> <title>The Cabin NYC - Google Maps</title> <meta content=" Find local businesses, view maps and get driving directions in Google Maps. " name="Description"/> <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-trial"/> <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/> <meta content="chrome=1" http-equiv="X-UA-Compatible"/> <meta content="notranslate" name="google"/> <meta content="origin" name="referrer"/> <meta content="ByHT0GXztW_RcGxS0o86DBf1WtNu02FfqlcT8njnSqU" name="google-site-verification"/> <meta 

In [83]:
for i in sp.find_all("div", {"class", "jftiEf fontBodyMedium"}):
    print(i.text)
    print("----------------------------------")

In [58]:
def scrape_google_maps_reviews(restaurant_name):
    s = Service(ChromeDriverManager().install())
    # Set up the Chrome WebDriver (replace 'path_to_chromedriver' with the actual path)
    driver = webdriver.Chrome(service=s)

    # Navigate to Google Maps
    driver.get('https://www.google.com/maps')

    # Search for the restaurant
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(restaurant_name)
    search_box.send_keys(Keys.ENTER)

    # Wait for the results to load
    time.sleep(3)

    # Find and click on the "More Reviews" button to load additional reviews
    try:
        while True:
            more_reviews_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(@aria-label, 'More reviews')]"))
            )
            more_reviews_button.click()
            time.sleep(2)  # Add a short delay after clicking the button to allow reviews to load
    except Exception as e:
        print("All reviews loaded.")
        print(e)
    
    # Scroll to load more reviews (adjust the number of scrolls as needed)
    num_scrolls = 10
    for _ in range(num_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

    # Extract reviews
    print(driver.page_source)
    reviews = driver.find_elements(By.XPATH, "//div[@class='section-review-content']//span[@class='section-review-text']")
    scraped_reviews = [review.text for review in reviews]

    # Close the browser
    driver.quit()

    if scraped_reviews:
        print("Scraped Reviews:")
    for idx, review in enumerate(scraped_reviews, start=1):
        print(f"{idx}. {review}")
    else:
        print("No reviews found or failed to scrape reviews.")
    return scraped_reviews


In [59]:
reviews = scrape_google_maps_reviews("The Cabin NYC")

All reviews loaded.
Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00E4A813+48355]
	(No symbol) [0x00DDC4B1]
	(No symbol) [0x00CE5358]
	(No symbol) [0x00D109A5]
	(No symbol) [0x00D10B3B]
	(No symbol) [0x00D3E232]
	(No symbol) [0x00D2A784]
	(No symbol) [0x00D3C922]
	(No symbol) [0x00D2A536]
	(No symbol) [0x00D082DC]
	(No symbol) [0x00D093DD]
	GetHandleVerifier [0x010AAABD+2539405]
	GetHandleVerifier [0x010EA78F+2800735]
	GetHandleVerifier [0x010E456C+2775612]
	GetHandleVerifier [0x00ED51E0+616112]
	(No symbol) [0x00DE5F8C]
	(No symbol) [0x00DE2328]
	(No symbol) [0x00DE240B]
	(No symbol) [0x00DD4FF7]
	BaseThreadInitThunk [0x7614FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77DC75F4+228]
	RtlGetAppContainerNamedObjectPath [0x77DC75C4+180]

<html itemscope="" itemtype="http://schema.org/Place" lang="en" jstcache="0"><head> <link href="/maps/preview/opensearch.xml?hl=en" title="Google Maps" rel="search" type="application/opensearchdescription+xml"> <title>The Cabin NYC - Google Map

In [47]:
if reviews:
        print("Scraped Reviews:")
        for idx, review in enumerate(reviews, start=1):
            print(f"{idx}. {review}")
else:
    print("No reviews found or failed to scrape reviews.")

No reviews found or failed to scrape reviews.


In [30]:
load_companies("https://www.google.com/maps/place/Berimbau+Brazilian+Kitchen/@40.729387,-74.0099787,16.25z/data=!4m6!3m5!1s0x89c25992f7b499a5:0x91869620d8f4ec19!8m2!3d40.7301198!4d-74.0036057!16s%2Fg%2F1tdk1htm?entry=ttu")

Getting business info https://www.google.com/maps/place/Berimbau+Brazilian+Kitchen/@40.729387,-74.0099787,16.25z/data=!4m6!3m5!1s0x89c25992f7b499a5:0x91869620d8f4ec19!8m2!3d40.7301198!4d-74.0036057!16s%2Fg%2F1tdk1htm?entry=ttu
<html itemscope="" itemtype="http://schema.org/Place" lang="en" jstcache="0"><head> <link href="/maps/preview/opensearch.xml?hl=en" title="Google Maps" rel="search" type="application/opensearchdescription+xml"> <title>Berimbau Brazilian Kitchen - Google Maps</title> <meta content=" Find local businesses, view maps and get driving directions in Google Maps. " name="Description"> <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-trial">  <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"> <met

Empty DataFrame
Columns: [Name, Rating, Reviews, Address, Category, Time]
Index: []

In [28]:
load_companies("https://www.google.com/maps/place/Berimbau+Brazilian+Kitchen/@40.729387,-74.0099787,16.25z/data=!4m6!3m5!1s0x89c25992f7b499a5:0x91869620d8f4ec19!8m2!3d40.7301198!4d-74.0036057!16s%2Fg%2F1tdk1htm?entry=ttu")

Getting business info https://www.google.com/maps/place/Berimbau+Brazilian+Kitchen/@40.729387,-74.0099787,16.25z/data=!4m6!3m5!1s0x89c25992f7b499a5:0x91869620d8f4ec19!8m2!3d40.7301198!4d-74.0036057!16s%2Fg%2F1tdk1htm?entry=ttu
<html itemscope="" itemtype="http://schema.org/Place" lang="en" jstcache="0"><head> <link href="/maps/preview/opensearch.xml?hl=en" title="Google Maps" rel="search" type="application/opensearchdescription+xml"> <title>Berimbau Brazilian Kitchen - Google Maps</title> <meta content=" Find local businesses, view maps and get driving directions in Google Maps. " name="Description"> <meta content="Anm+hhtuh7NJguqSnXHEAIqqMaV+GXCks8WYXHJKF7l6AeYMj+wO+fi9OdDqFnJTg9t0492DykVxx4jpvFbxnA8AAABseyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IlByaXZhY3lTYW5kYm94QWRzQVBJcyIsImV4cGlyeSI6MTY5NTE2Nzk5OSwiaXNTdWJkb21haW4iOnRydWV9" http-equiv="origin-trial">  <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"> <met

Empty DataFrame
Columns: [Name, Rating, Reviews, Address, Category, Time]
Index: []